## Задание 1.
Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

In [1]:
# Импортируем нужные библиотеки
import requests, json
from lxml import html

In [2]:
# Прописываем константы
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
URL_DZEN = 'https://dzen.ru/news'

In [3]:
# Функция, позволяющая считать данные страницы и преобразовать их в DOM
def get_dom_from_url(url):
    response = requests.get(url, headers = {'User-Agent': USER_AGENT}, params={'sso_failed': ''})
    dom = html.fromstring(response.text)
    return dom

In [4]:
# Функция, позволяющая спарсить из полученного массива необходимые данные
def get_news_dzen(dom):
    dzen_news = []
# В Дзен новости разбиты по областям, считываем сначала их
    fields = dom.xpath('//section[@aria-labelledby]')
    for field in fields:
# Далее уже в каждом разделе считываем отдельные новости
        news = field.xpath('.//div[contains(@class,"mg-grid__col")]')
        for new in news:
# Решил добавить помимо запрошенных данных область новости
            if field.xpath('.//@aria-labelledby')[0] == 'top-heading':
                new_field = field.xpath('.//h1/text()')[0]
            else:
                new_field = field.xpath('.//h1/a/text()')[0]
            new_source = new.xpath('.//span[@class="mg-card-source__source"]/a/text()')[0]
            new_link = new.xpath('.//span[@class="mg-card-source__source"]/a/@href')[0]
            new_text = new.xpath('.//h2[@class="mg-card__title"]/a/text()')[0].replace('\xa0', ' ')
            new_time = new.xpath('.//span[@class="mg-card-source__time"]/text()')[0]
            dzen_news.append({'new_field': new_field,
                              'new_source': new_source,
                              'new_text': new_text,
                              'new_link': new_link,
                              'new_time': new_time})
    return(dzen_news)

In [5]:
# Вызываем функции
dom = get_dom_from_url(URL_DZEN)
news = get_news_dzen(dom)
news

[{'new_field': 'Главное',
  'new_source': 'Lenta.ru',
  'new_text': 'Мэр Енакиево Храменков сообщил, что над территорией города был сбит самолет',
  'new_link': 'https://dzen.ru/news/story/Mehr_Enakievo_KHramenkov_soobshhil_chto_nadterritoriej_goroda_byl_sbit_samolet--1476de7d07400648eeea2798723b3992?lang=ru&rubric=index&fan=1&stid=C2RIPTBj9kDsQXfIncH5&t=1677859894&tt=true&persistent_id=2253360001&story=022f378e-2255-5da2-8809-ebd282df41d5&issue_tld=ru',
  'new_time': '18:58'},
 {'new_field': 'Главное',
  'new_source': 'Lenta.ru',
  'new_text': 'РИА Новости: в Москве убили одного из разработчиков вакцины «Спутник V» Андрея Ботикова',
  'new_link': 'https://dzen.ru/news/story/RIA_Novosti_vMoskve_ubili_odnogo_izrazrabotchikov_vakciny_Sputnik_V_Andreya_Botikova--371d05618a1de3ee745c12e1183559b8?lang=ru&rubric=index&fan=1&stid=TjJgYxRKswL4BDKH7Tjo&t=1677859894&tt=true&persistent_id=2247760408&story=0e5b0d3a-ae6e-5f4a-bb41-bc87d3796570&issue_tld=ru',
  'new_time': '19:10'},
 {'new_field': '